***
# <font color=red>Building a Forecaster using AutoMLx</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle AutoMLx Team </font></p>

***

AutoMLx Forecasting Demo version 23.1.1.

Copyright (c) 2023 Oracle, Inc.  

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

## Overview of this Notebook

In this notebook we will build a forecaster using the Oracle AutoMLx tool for three real-world datasets. We explore the various options available in the Oracle AutoMLx Forecasting module, allowing the user to control the AutoML training process. We finally evaluate the statistical forecasting algorithms using in-built visualization tools. Note that contrary to other tasks like Classification, Regression or Anomaly detection, the AutoML package does not yet support explainability for forecasting tasks.

---
## Prerequisites

  - Experience level: Novice (Python and Machine Learning)
  - Professional experience: Some industry exprience

Compatible conda pack: [Oracle AutoML and Model Explanation for Python 3.8 (version 2.0)](oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/Oracle_AutoML_and_Model_Explanation_for_Python_3.8/2.0/automlx_p38_cpu_v2)

---

## Business Use

Forecasting uses historical time series data as input to make informed estimates of future trends. Learning accurate statistical forecasting model requires expertise in data science and statistics. This process typically comprises of: 
- Preprocess dataset (clean, impute, engineer features, normalize).
- Pick an appropriate model for the given dataset and prediction task at hand.
- Tune the chosen model’s hyperparameters for the given dataset.

These steps are significantly time consuming and heavily rely on data scientist expertise. Unfortunately, to make this problem harder, the best feature subset, model, and hyperparameter choice widely varies with the dataset and the prediction task. Hence, there is no one-size-fits-all solution to achieve reasonably good model performance. Using a simple Python API, AutoML can quickly (faster) jump-start the datascience process with an accurately-tuned model and appropriate features for a given prediction task.

## Table of Contents

- <a href='#setup'>0. Setup</a>
- <a href='#univariate'>1. Univariate time series
    - <a href='#load-m4'>1.1. Load the M4 Forecasting Competition dataset
    - <a href='#task'>1.2. Split data into train and test for the forecasting task</a>
    - <a href='#Engine'>1.3. Set the AutoML engine</a>
    - <a href='#provider'>1.4. Create an instance of Oracle AutoMLx</a>
    - <a href='#default'>1.5. Train a forecasting model using AutoMLx</a>
    - <a href='#forecast'>1.6. Generate and visualize forecasts</a>
    - <a href='#analysis'>1.7. Analyze the AutoML optimization process</a>
        - <a href='#algorithm-selection'>1.7.1 Algorithm Selection</a>
        - <a href='#hyperparameter-tuning'>1.7.2 Hyperparameter Tuning</a>
    - <a href='#load-data-air'>1.8. Load the Airline Dataset</a>
    - <a href='#scoringstr'>1.9. Specify a different score metric for optimization</a>
    - <a href='#WFCV'>1.10. Specify the number of cross-validation (CV) folds</a>
- <a href='#multi'>2. Multivariate time series</a>
    - <a href='#multi-generating'>2.1. Generate the data</a>
    - <a href='#multi-fitting'>2.2. Train a model using Oracle AutoMLx</a>
    - <a href='#multi-making'>2.3. Make predictions</a>
    - <a href='#multi-visualization'>2.4. Visualization</a>
- <a href='#ref'>References</a>

<a id='setup'></a>
## Setup

Basic setup for the Notebook.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

Load the required modules.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import gzip
import matplotlib.pyplot as plt
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.datasets import load_airline

plt.rcParams['figure.figsize'] = [15, 5]
plt.rcParams['font.size'] = 15
sns.set(color_codes=True)
sns.set(font_scale=1.5)
sns.set_palette("bright")
sns.set_style("whitegrid")

import automl
from automl import init
from automl.interface.utils import plot_forecast

<a id='univariate'></a>
# Univariate time series
The Oracle AutoMLx solution for forecasting can process both univariate and multivariate time series. We start by displaying an example of use for univariate time series, and will adress multivariate data at the end of this notebook.<br> 

<a id='load-m4'></a>
### Load the M4 Forecasting Competition dataset 


We fetch a  univariate timeseries from the repository of the [M4 forecasting competition](https://mofc.unic.ac.cy/m4/).

In [ ]:
m4_url = "https://github.com/Mcompetitions/M4-methods/raw/master/Dataset/Train/Weekly-train.csv"
m4_metadata_url = "https://github.com/Mcompetitions/M4-methods/raw/master/Dataset/M4-info.csv"

all_series = pd.read_csv(m4_url, index_col=0)  # consists of thousands of series
metadata_csv = pd.read_csv(m4_metadata_url, index_col=0)  # describes their datetime index

We select a series from the finance sector with weekly collection frequency. M4 dataset requires additional preprocessing to reconstruct the timeseries.

In [ ]:
series_id = 'W142'
series_metadata = metadata_csv.loc[series_id]
series_values = all_series.loc[series_id]

# drop NaNs for the time period where data wasn't recorded
series_values.dropna(inplace=True)

# retrieve starting date of recording and series length to generate the datetimeindex
start_date = pd.to_datetime(series_metadata.StartingDate)
future_dates = pd.date_range(start=start_date,
                             periods=len(series_values),
                             freq='W', closed=None)
y = pd.DataFrame(series_values.to_numpy(),
                 index=future_dates,
                 columns=[(series_metadata.category+"_"+series_id)])

We can now visualize the last 200 weeks of data we have on hand.

In [ ]:
y = y.tail(n=200)  # approximately 4 years of data
y.plot(ylabel='Weekly Series '+series_id, grid=True)

One must ensure that the data points are in a Pandas DataFrame, sorted in chronological order.

In [ ]:
print(y.index)
print("Time Index is", "" if y.index.is_monotonic else "NOT", "monotonic.")
print("Train datatype", type(y))

<a id='task'></a>
### Split data into train and test for the forecasting task
As can be seen above, the data contains 200 weekly recorded values over the past 5 years. We will try to predict electricity consumption for the last 0.5 year of data (26 data points), using the previous years as training data. Hence, we separate the dataset into training and testing sets using Temporal train-test split, which ensures that the continuity of the input time series is preserved. Each point in the series represents a month, so we will hold out the last 26 points as test data.

In [ ]:
y_train, y_test = temporal_train_test_split(y, test_size=26)
print("Training length: ", len(y_train)," Testing length: ", len(y_test))

We see that the train data ranges from December 2012 to April 2016, while the test data ranges from April to October 2016.

In [ ]:
print("y_train", y_train)
print("\ny_test", y_test)

<a id='Engine'></a>
### Set the AutoML engine
The AutoML pipeline offers the function `init`, which allows to initialize the parallel engine. By default, the AutoML pipeline uses the `dask` parallel engine. One can also set the engine to `local`, which uses python's multiprocessing library for parallelism instead.

In [ ]:
init(engine='local')

<a id='provider'></a>
### Create an instance of Oracle AutoMLx

The Oracle AutoMLx solution automatically provides a tuned forecasting pipeline that best models the given training dataset and a prediction task at hand. Here the dataset can be any univariate time-series.

AutoML for Forecasting consists of three main modules: 
- **Preprocessing**
    - Impute any missing values using back fill or forward fill mechanisms to ensure input has a well-defined and consistent frequency.
    - Identify seasonalities present in the data by detrending and analyzing the Autocorrelation Function (ACF) of the series.
    - Decide appropriate number of cross-validation (CV) folds and the forecast horizons based on the datetime frequency of data.
- **Algorithm Selection**: Identify the right algorithm for a given dataset, choosing from the following:
    - NaiveForecaster - Naive and Seasonal Naive method
    - ThetaForecaster - Equivalent to Simple Exponential Smoothing (SES) with drift
    - ExpSmoothForecaster - Holt-Winters' damped method
    - STLwESForecaster - Seasonal Trend LOESS (locally weighted smoothing) with Exponential Smoothing substructure
    - STLwARIMAForecaster - Seasonal Trend LOESS (locally weighted smoothing) with ARIMA substructure
    - SARIMAForecaster - Seasonal Autoregressive Integrated Moving Average
    - ETSForecaster - Error, Trend, Seasonality (ETS) Statespace Exponential Smoothing
    - ProphetForecaster (optional) - Facebook Prophet. Only available if installed locally with `pip install fbprophet`
    - OrbitForecaster (optional) - Uber Orbit model with Exogenous Variables. (Available if a supported version is installed)
    - VARMAXForecaster - Vector AutoRegressive Moving Average with Exogenous Variables (Available for multivariate datasets)
    - DynFactorForecaster - Dynamic Factor Models in state-space form with Exogenous Variables (Available for multivariate datasets)
- **Hyperparameter Tuning** 
    - Find the right model parameters that maximize score for the given dataset. 

These pieces are readily combined into a simple AutoML pipeline which automates the entire forecasting process with minimal user input/interaction. One can then evaluate and visualize the forecast produced by the selected model, and optionally the other tuned models.

<a id='default'></a>
### Train a forecasting model using Oracle AutoMLx

The AutoML API is quite simple to work with. We first create an instance of the pipeline. Next, the training data is passed to the `fit()` function which successively executes the previously mentioned modules.

The generated model can then be used for forecasting tasks. By default, we use the negative of symmetric mean absolute percentage error (sMAPE)  scoring metric to evaluate the model performance. The parameter `n_algos_tuned` sets the number of algorithms whose hyperparameters are fully tuned. For highest accuracy results, it is recommended to set this value to >=2 and preferably to 8 such that all models are fully tuned.

In [ ]:
est1 = automl.Pipeline(task='forecasting', n_algos_tuned=4)
est1.fit(X=None, y=y_train)

print('Selected model: {}'.format(est1.selected_model_))
print('Selected model params: {}'.format(est1.selected_model_params_))

The selected model params indicate a good fit at sp (seasonal periodicity) of 52, which typically corresponds to a yearly seasonality for data that is weekly collected (i.e., there are 52 weeks in a year).

<a id='forecast'></a>
### Generating and visualizing forecasts
There are two interfaces that support generating future forecasts using the trained forecasting pipeline.
The preferred function, `forecast()`, accepts a user-input value for the number of periods to forecast into the future, i.e., relative to the end of the training series. It also accepts a significance level to generate prediction confidence intervals (CIs). When the methods support intervals, confidence intervals at 1-alpha are generated, e.g., significance level alpha=0.05 generates 95% confidence intervals.

In [ ]:
summary_frame = est1.forecast(periods=len(y_test), alpha=0.05)
print(summary_frame)

The `predict(X)` interface supports absolute index-based forecasts, but does not support confidence intervals (CIs). It also downcasts the index to int64. 

It should be utilized only when you want to get both in-sample predictions (predictions at timestamps that are part of the training set) and out-of-sample predictions (predictions at timestamps that are not in the training set). The X variable should be an empty dataframe containing only the requested timestamps as index. Here we request 5 in-sample model fit values and 5 out-of-sample forecasts.

In [ ]:
future_index = y_train.index[-5:].union(y_test.index[:5])
print(future_index)

In [ ]:
est1.predict(X=pd.DataFrame(index=future_index))

AutoML provides a simple one-line tool to visualize forecasts and confidence intervals.

In [ ]:
automl.interface.utils.plot_forecast(fitted_pipeline=est1, summary_frame=summary_frame, 
                                           additional_frames=dict(y_test=y_test)) 

<a id='analysis'></a>
### Analyze the AutoML optimization process
During the AutoML process, a summary of the optimization process is logged. It consists of:
- Information about the training data 
- Information about the AutoML Pipeline, such as:
    - selected features that AutoML found to be most predictive in the training data;
    - selected algorithm that was the best choice for this data;
    - hyperparameters for the selected algorithm.

AutoML provides a `print_summary()` API to output all the different trials performed.

In [ ]:
est1.print_summary()

We also provide the capability to visualize the results of each stage of the AutoML pipeline. 

<a id='algorithm-selection'></a>
#### Algorithm Selection

The plot below shows the scores predicted by Algorithm Selection for each algorithm. Since negative sMAPE is used by default, higher values (closer to zero) are better. The horizontal line shows the average score across all algorithms. Algorithms with better score than average are colored turquoise, whereas those with worse score than average are colored teal. Here we can see that the `SARIMAXForecaster` algorithm achieved the best predicted score (orange bar), and is chosen for subsequent stages of the Pipeline.

In [ ]:
# Each trial is a tuple of
# (algorithm, no. samples, no. features, mean CV score, hyperparameters, 
# all CV scores, total CV time (s), memory usage (Gb))
trials = est1.model_selection_trials_ 
scores = [x[3] for x in trials]
models = [x[0] for x in trials]
y_margin = 0.10 * (max(scores) - min(scores))
s = pd.Series(scores, index=models).sort_values(ascending=False)

colors = []
for f in s.keys():
    if f == '{}_AS'.format(est1.selected_model_):
        colors.append('orange')
    elif s[f] >= s.mean():
        colors.append('teal')
    else:
        colors.append('turquoise')
        

fig, ax = plt.subplots(1)
ax.set_title("Algorithm Selection Trials")
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.set_ylabel(est1.inferred_score_metric[0])
s.plot.bar(ax=ax, color=colors, edgecolor='black')
ax.axhline(y=s.mean(), color='black', linewidth=0.5)
plt.show()

<a id='hyperparameter-tuning'></a>
#### Hyperparameter Tuning

Hyperparameter tuning is the last stage of the Oracle AutoMLx pipeline, and focuses on improving the chosen algorithm's score. We use a novel algorithm to search across many hyperparameter dimensions, and converge automatically when optimal hyperparameters are identified. Each trial in the graph below represents a particular hyperparameter combination for the selected model.

In [ ]:
# Each trial is a tuple of
# (algorithm, no. samples, no. features, mean CV score, hyperparameters, 
# all CV scores, total CV time (s), memory usage (Gb))
trials = np.array(est1.tuning_trials_)
scores = np.array([x[3] for x in reversed(trials)])
finite_indexes = np.where(np.isfinite(scores))

trials = trials[finite_indexes]
scores = scores[finite_indexes]
y_margin = 0.10 * (max(scores) - min(scores))


fig, ax = plt.subplots(1)
ax.set_title("Hyperparameter Tuning Trials")
ax.set_xlabel("Iteration $n$")
ax.set_ylabel(est1.inferred_score_metric[0])
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(range(1, len(trials) + 1), scores, 'k:', marker="s", color='teal', markersize=3)
plt.show()

We can also view all tuned algorithms, as well as their validation and testing performance. This provides a good sanity check for the decision making.

In [ ]:
print(f'### Plotting is enabled for total models tuned = {len(est1.pipelines_)}.')
print('### Model_name\t\t Val_score\t Test_score ')
for i in range(0, len(est1.pipelines_)):
    pipe_ = est1.pipelines_[i]
    print(  pipe_.__dict__['selected_model_'] , " \t",
            '%.4f'%pipe_.k_results[pipe_.__dict__['selected_model_']]['best_score'],'\t',   # validation_score
            '%.4f'%pipe_.score(pd.DataFrame(index=y_test.index), y=y_test))            # testing_score
    summary_frame = pipe_.forecast(len(y_test), alpha=0.05)                     # out-of-sample forecast
    fig = automl.interface.utils.plot_forecast(fitted_pipeline=pipe_, summary_frame=summary_frame, 
                                               additional_frames=dict(y_text=y_test))
    fig.show()

<a id='load-data-air'></a>
### Load the Airline Dataset
The  Airline  Passenger univariate series represents  the  monthly  total  number  of  international airline passengers (in thousands) from January 1949 to December 1960. To showcase AutoML's functionality in the absence of a datetime index, we drop the datetime index and utilize the series with only an int64index.

In [ ]:
yair = pd.DataFrame(load_airline()) # Input must be a pd.DataFrame type
yair.index = np.arange(0, len(yair)) # replace the datetime index with an integer index.
yair.plot(ylabel='Number of Airline Passengers', grid=True)

We will forecast the last 20% of data, using the previous years as training data. 

In [ ]:
yair_train, yair_test = temporal_train_test_split(yair, test_size=0.2)
print("Training length: ", len(yair_train)," Testing length: ", len(yair_test))

<a id='scoringstr'></a>
### Specify a different score metric for Oracle AutoMLx optimization
The pipeline tries to maximize a given score metric, by looking at different methods and hyperparameter choices. By default, the score metric is set to negative of sMAPE. The user can also choose another metric. For the forecasting task, possible metrics are:
'neg_sym_mean_abs_percent_error', 'neg_root_mean_squared_percent_error', 'neg_mean_abs_scaled_error', 
                            'neg_root_mean_squared_error', 'neg_mean_squared_error', 'neg_max_absolute_error', 'neg_mean_absolute_error' is accepted.


Here, we ask AutoML to optimize for MASE ('neg_mean_abs_scaled_error'), a scale-invariant scoring metric.

In [ ]:
est2 = automl.Pipeline(task='forecasting', n_algos_tuned=3, score_metric='neg_mean_abs_scaled_error')
est2.fit(y=yair_train)

test_score = automl.models.score.time_series_loss(est2, X=pd.DataFrame(index=yair_test.index), 
                                                  y=yair_test, scoring='neg_mean_abs_scaled_error')
print('Selected model: {}'.format(est2.selected_model_))
print('Selected model params: {}'.format(est2.selected_model_params_))
print(f'Score on test data : {test_score}')

In [ ]:
automl.interface.utils.plot_forecast(fitted_pipeline=est2, summary_frame=est2.forecast(len(yair_test)), 
                                           additional_frames=dict(y_test=yair_test, y_train=yair_train))

<a id='WFCV'></a>
### Specify the number of cross-validation (CV) folds
AutoML automatically decides how many folds to create, given the length of the input series. This is dependent on the frequency and length of the series. 
In the above, the preprocessor chose to create two folds. In the following we set the number of folds to 8.

In [ ]:
est3 = automl.Pipeline(task='forecasting')
est3.fit(y=yair_train, cv=8)

print('Selected model: {}'.format(est3.selected_model_))
print('Selected model params: {}'.format(est3.selected_model_params_))
print(f'Score on test data : {est3.score(pd.DataFrame(index=yair_test.index), y=yair_test)}')

fig = automl.interface.utils.plot_forecast(fitted_pipeline=est3, summary_frame=est3.forecast(len(yair_test)), 
                                           additional_frames=dict(y_test=yair_test))

<a id='multi'></a>
## Multivariate time series

<a id='multi-generating'></a>
### Generate the data

We now display the use of the Oracle AutoMLx solution for multivariate timeseries. We load the 10-dimensional Lutkepohl2 dataset. We then restrict the data to 4 variables : two exogenous variables (variables that are independent on all other data variables), and two endogenous variables (variables that are dependent on some other data variables). The endogenous variables will be the target predictions of the pipeline, while the exogenous variables will be used solely as explanatory variables. 

In [ ]:
dta = sm.datasets.webuse('lutkepohl2', 'https://www.stata-press.com/data/r12/')
dta.index = dta.qtr
dta.index.freq = dta.index.inferred_freq
endog = dta.loc['1960-04-01':'1978-10-01', ['dln_inv', 'dln_inc',]]
exog = dta.loc['1960-04-01':'1978-10-01', ['dln_consump']]
exog = sm.add_constant(exog)

We then split it using a temporal train-test split as done previously. Note that $X$ consists of the exogenous variables ('dln_consump' and another constant variable) while $y$ are the target variables ('dln_in' and 'dln_inc')

In [ ]:
X_train_df, X_test_df = temporal_train_test_split(exog, train_size=0.9)
y_train_df, y_test_df = temporal_train_test_split(endog, train_size=0.9)

<a id='multi-fitting'></a>
### Train a model using Oracle AutoMLx

We can now fit the AutoML pipeline. For the multivariate forecasting task, the pipeline only considers two models : `VARMAX` and `DynFactor`.

In [ ]:
pipeline = automl.Pipeline(task='forecasting',
                           n_algos_tuned=1,
                           score_metric='neg_sym_mean_abs_percent_error')

pipeline.fit(X=X_train_df, y=y_train_df)

The AutoML pipeline provides attributes to get the selected features, the chosen model, hyperparameters as well as the score on the test set.

In [ ]:
print('Selected features: {}'.format(pipeline.selected_features_))
print('Ranked models: {}'.format(pipeline.ranked_models_))
print('Selected model: {}'.format(pipeline.selected_model_))
print('Selected model params: {}'.format(pipeline.selected_model_params_))

<a id='multi-making'></a>
### Make predictions

As mention in the univariate data case, there are two ways of making a prediction : 
- `forecast(k)` allows one to predict k steps after the end of the training data. It should be used when one wants to make out-of-sample predictions
- `predict(X)` returns predictions at the timestamps given as argument. It should be used when one wants to make in-sample predictions and out-of-sample predictions. It does not support confidence intervals.

In the cell below `predict()` is used on the last 5 timestamps of the train set, and all timestamps of the test set. `forecast()` is used to predict k steps after the training set, where k is the size of the test set.

In [ ]:
y_pred = pipeline.predict(pd.concat([X_train_df[-5:0],X_test_df], axis=0) )
y_forecast = pipeline.forecast(len(y_test_df), alpha=0.8, X=X_test_df)    # out-of-sample forecast

The obtained forecast contains predictions for the two target variables, as well as lower and upper confidence intervals, for each timestamp in the test set.

In [ ]:
y_forecast

One can also directly compute the score of the tuned model on the test set, without needing to run `forecast()` or `predict()`.

In [ ]:
print("Tuned model testing score (negative sMAE): ", pipeline.score(X=X_test_df, y=y_test_df))

<a id='multi-visualization'></a>
### Visualization

Finally, when given as input the forecasted variables, the `plot_forecast()` method displays an interactive plot of the predictions (for each target variable) and confidence intervals.

In [ ]:
plot_forecast(fitted_pipeline=pipeline, summary_frame=y_forecast, additional_frames=dict(test=y_test_df))

<a id='ref'></a>
## References
* More examples and details: http://automl.oraclecorp.com/
* Oracle AutoML: http://www.vldb.org/pvldb/vol13/p3166-yakovlev.pdf
* sktime: https://www.sktime.org/en/latest/
* statsmodels: https://www.statsmodels.org/stable/index.html
* M4 Competition: https://mofc.unic.ac.cy/m4/
* Airline Dataset: https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.datasets.load_airline.html
